In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# Set the paths for the directories
base_dir = 'C:\\Users\\hp\\Contacts\\pythonProject\\input_datasets\\animals\\animals'



In [5]:
def load_data_batch(base_dir, batch_size, start_idx):
    data = []
    labels_type = []
    labels_name = []

    animal_types = ['herbivorous', 'carnivorous']
    animal_type_map = {animal_type: idx for idx, animal_type in enumerate(animal_types)}

    herbivores = [
        'antelope', 'bison', 'butterfly', 'cow', 'deer', 'donkey', 'elephant',
        'flamingo', 'goat', 'gorilla', 'grasshopper', 'hare', 'hippopotamus',
        'horse', 'kangaroo', 'koala', 'ladybugs', 'moth', 'mouse', 'okapi',
        'ox', 'panda', 'parrot', 'pig', 'porcupine', 'possum', 'rabbit', 'rat',
        'reindeer', 'rhinoceros', 'sandpiper', 'sheep', 'squirrel', 'swan',
        'turtle', 'wombat', 'zebra'
    ]

    carnivores = [
        'badger', 'bat', 'bear', 'cat', 'chimpanzee', 'coyote', 'crab', 'crow',
        'dog', 'dolphin', 'eagle', 'fox', 'goldfish', 'hyena', 'leopard', 'lion',
        'lobster', 'octopus', 'orangutan', 'otter', 'owl', 'shark', 'snake', 'sparrow',
        'squid', 'starfish', 'tiger', 'whale', 'wolf', 'woodpecker'
    ]

    animal_names = herbivores + carnivores
    animal_name_map = {animal_name: idx for idx, animal_name in enumerate(animal_names)}

    all_images = []

    for animal_type in animal_types:
        type_path = os.path.join(base_dir, animal_type)
        for animal_name in os.listdir(type_path):
            animal_path = os.path.join(type_path, animal_name)
            for img_file in os.listdir(animal_path):
                img_path = os.path.join(animal_path, img_file)
                all_images.append((img_path, animal_type, animal_name))

    batch_images = all_images[start_idx:start_idx + batch_size]

    for img_path, animal_type, animal_name in batch_images:
        img = load_img(img_path, target_size=(150, 150))
        img_array = img_to_array(img)
        data.append(img_array)
        labels_type.append(animal_type_map[animal_type])
        labels_name.append(animal_name_map[animal_name])

    data = np.array(data, dtype="float32") / 255.0
    labels_type = np.array(labels_type)
    labels_name = np.array(labels_name)

    return data, labels_type, labels_name

def get_total_images(base_dir):
    total_images = 0
    for animal_type in ['herbivorous', 'carnivorous']:
        type_path = os.path.join(base_dir, animal_type)
        for animal_name in os.listdir(type_path):
            animal_path = os.path.join(type_path, animal_name)
            total_images += len(os.listdir(animal_path))
    return total_images

total_images = get_total_images(base_dir)
batch_size = 256  # Adjust batch size as needed


In [6]:
animal_types = ['herbivorous', 'carnivorous']
herbivores = [
     'butterfly','chiken' ,'cow',  'elephant',
    'horse', 'sheep', 'squirrel'
    ]

carnivores = [
    'cat', 'dog', 'spider'
    ]

animal_names = herbivores + carnivores
input_shape = (150, 150, 3)
num_animal_types = len(animal_types)
num_animal_names = len(animal_names)

inputs = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

type_output = Dense(num_animal_types, activation='softmax', name='animal_type')(x)
name_output = Dense(num_animal_names, activation='softmax', name='animal_name')(x)

model = Model(inputs=inputs, outputs=[type_output, name_output])


In [8]:
model.compile(
    optimizer='adam',
    loss={
        'animal_type': 'sparse_categorical_crossentropy',
        'animal_name': 'sparse_categorical_crossentropy'
    },
    metrics={
        'animal_type': 'accuracy',
        'animal_name': 'accuracy'
    }
)


In [9]:
from tqdm import tqdm

epochs = 18
batch_size = 256
total_images = get_total_images(base_dir)
steps_per_epoch = total_images // batch_size

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    progress_bar = tqdm(total=steps_per_epoch, desc="Training")

    for start_idx in range(0, total_images, batch_size):
        data_batch, labels_type_batch, labels_name_batch = load_data_batch(base_dir, batch_size, start_idx)
        loss = model.train_on_batch(data_batch, {'animal_type': labels_type_batch, 'animal_name': labels_name_batch})
        progress_bar.update(1)
        progress_bar.set_postfix({
            'loss': loss[0],
            'animal_type_loss': loss[1],
            'animal_name_loss': loss[2],
            'animal_type_acc': loss[3],
            'animal_name_acc': loss[4]
        })
    
    progress_bar.close()


Epoch 1/18


Training: 16it [01:50,  6.94s/it, loss=4.88, animal_type_loss=0.668, animal_name_loss=4.22, animal_type_acc=1, animal_name_acc=0]                                 


Epoch 2/18


Training: 16it [01:31,  5.75s/it, loss=4.91, animal_type_loss=0.694, animal_name_loss=4.21, animal_type_acc=0, animal_name_acc=0]                           


Epoch 3/18


...

Training: 16it [01:30,  5.64s/it, loss=4.87, animal_type_loss=0.7, animal_name_loss=4.17, animal_type_acc=0, animal_name_acc=0.575]                         


Epoch 17/18


Training: 16it [01:29,  5.62s/it, loss=4.87, animal_type_loss=0.701, animal_name_loss=4.17, animal_type_acc=0, animal_name_acc=0.592]                          


Epoch 18/18


Training: 16it [01:42,  6.42s/it, loss=4.86, animal_type_loss=0.7, animal_name_loss=4.16, animal_type_acc=0.05, animal_name_acc=0.533]                           


In [7]:
model.save('animals (1).h5')

c:\Users\hp\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
from tensorflow.keras.preprocessing import image

def load_and_preprocess_image(img_path, target_size=(150, 150)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Rescale
    return img_array


In [9]:
def predict_animal(model, img_path, animal_name_map, animal_type_map):
    # Load and preprocess the image
    img_array = load_and_preprocess_image(img_path)
    
    # Make predictions
    predictions = model.predict(img_array)
    type_prediction = np.argmax(predictions[0], axis=1)[0]
    name_prediction = np.argmax(predictions[1], axis=1)[0]

    # Map predictions to labels
    inv_animal_type_map = {v: k for k, v in animal_type_map.items()}
    inv_animal_name_map = {v: k for k, v in animal_name_map.items()}

    predicted_type = inv_animal_type_map[type_prediction]
    predicted_name = inv_animal_name_map[name_prediction]

    return predicted_type, predicted_name
